In [3]:
import pandas as pd
import numpy as np

from sklearn2.model_selection import KFold
from sklearn2.neural_network import MLPRegressor
from sklearn2.model_selection import RandomizedSearchCV
from sklearn2.metrics import make_scorer
from sklearn2 import preprocessing

from math import radians, cos, sin, asin, sqrt

from datetime import datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
pd.options.display.float_format = "{:.18f}".format

In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191392039,40.767936706542968750,-73.964630126953125000,40.765602111816406250,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238267039,40.738563537597656250,-73.999481201171875000,40.731151580810546875,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433579539,40.763938903808593750,-74.005332946777343750,40.710086822509772730,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125000,40.719970703125000000,-74.012268066406250000,40.706718444824218750,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625000,40.793209075927727270,-73.972923278808593750,40.782520294189453125,N,435


In [7]:
# quitar valores vacíos
df.dropna(inplace=True)

In [8]:
mean, std_deviation = np.mean(df['trip_duration']), np.std(df['trip_duration'])
df = df[df['trip_duration'] <= mean + 2 * std_deviation]
df = df[df['trip_duration'] >= mean - 2 * std_deviation]

In [9]:
# crear variables más cortas para las columnas con nombres largos
plg, plt = 'pickup_longitude', 'pickup_latitude'
dlg, dlt = 'dropoff_longitude', 'dropoff_latitude'
pdt, ddt = 'pickup_datetime', 'dropoff_datetime'

In [10]:
# función para calcular la distancia desde la recogida hasta la llegada
def haversine(lon1, lat1, lon2, lat2):
    # convertir grados decimal a radianes 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # formula haversine 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # radio de la tierra en kilometros es 6371
    km = 6371* c
    return km

def euclidian_distance(x):
    x1, y1 = np.float64(x[plg]), np.float64(x[plt])
    x2, y2 = np.float64(x[dlg]), np.float64(x[dlt])    
    return haversine(x1, y1, x2, y2)

In [11]:
# crear una columna con la distancia calculada desde la recogida hasta la llegada
df['distance'] = df[[plg, plt, dlg, dlt]].apply(euclidian_distance, axis=1)
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191392039,40.767936706542968750,-73.964630126953125000,40.765602111816406250,N,455,1.498520779645855727
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238267039,40.738563537597656250,-73.999481201171875000,40.731151580810546875,N,663,1.805507168796843587
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433579539,40.763938903808593750,-74.005332946777343750,40.710086822509772730,N,2124,6.385098495252204742
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125000,40.719970703125000000,-74.012268066406250000,40.706718444824218750,N,429,1.485498422770938243
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625000,40.793209075927727270,-73.972923278808593750,40.782520294189453125,N,435,1.188588459333177694


In [12]:
# convertir de string a datetime
df[pdt] = df[pdt].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df[ddt] = df[ddt].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [13]:
# crear columnas a partir de la hora de recogida
df['month'] = df[pdt].apply(lambda x : x.month)
df['weekDay'] = df[pdt].apply(lambda x : x.weekday())
df['dayMonth'] = df[pdt].apply(lambda x : x.day)
df['pickupTimeMinutes'] = df[pdt].apply(lambda x : x.hour * 60.0 + x.minute)
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,month,weekDay,dayMonth,pickupTimeMinutes
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191392039,40.767936706542968750,-73.964630126953125000,40.765602111816406250,N,455,1.498520779645855727,3,0,14,1044.000000000000000000
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238267039,40.738563537597656250,-73.999481201171875000,40.731151580810546875,N,663,1.805507168796843587,6,6,12,43.000000000000000000
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433579539,40.763938903808593750,-74.005332946777343750,40.710086822509772730,N,2124,6.385098495252204742,1,1,19,695.000000000000000000
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125000,40.719970703125000000,-74.012268066406250000,40.706718444824218750,N,429,1.485498422770938243,4,2,6,1172.000000000000000000
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625000,40.793209075927727270,-73.972923278808593750,40.782520294189453125,N,435,1.188588459333177694,3,5,26,810.000000000000000000


In [14]:
# eliminar columnas innecesarias
df.drop(['id', pdt, ddt, dlg, dlt, 'store_and_fwd_flag'], inplace=True, axis=1)
df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,trip_duration,distance,month,weekDay,dayMonth,pickupTimeMinutes
0,2,1,-73.982154846191392039,40.767936706542968750,455,1.498520779645855727,3,0,14,1044.000000000000000000
1,1,1,-73.980415344238267039,40.738563537597656250,663,1.805507168796843587,6,6,12,43.000000000000000000
2,2,1,-73.979026794433579539,40.763938903808593750,2124,6.385098495252204742,1,1,19,695.000000000000000000
3,2,1,-74.010040283203125000,40.719970703125000000,429,1.485498422770938243,4,2,6,1172.000000000000000000
4,2,1,-73.973052978515625000,40.793209075927727270,435,1.188588459333177694,3,5,26,810.000000000000000000


In [15]:
# reordenar columnas
df = df[[plg, plt, 'distance', 'month', 'dayMonth', 'weekDay', 'pickupTimeMinutes', 'passenger_count', 'vendor_id', 'trip_duration']]
df.head()

,pickup_longitude,pickup_latitude,distance,month,dayMonth,weekDay,pickupTimeMinutes,passenger_count,vendor_id,trip_duration
0,-73.982154846191392039,40.767936706542968750,1.498520779645855727,3,14,0,1044.000000000000000000,1,2,455
1,-73.980415344238267039,40.738563537597656250,1.805507168796843587,6,12,6,43.000000000000000000,1,1,663
2,-73.979026794433579539,40.763938903808593750,6.385098495252204742,1,19,1,695.000000000000000000,1,2,2124
3,-74.010040283203125000,40.719970703125000000,1.485498422770938243,4,6,2,1172.000000000000000000,1,2,429
4,-73.973052978515625000,40.793209075927727270,1.188588459333177694,3,26,5,810.000000000000000000,1,2,435


In [16]:
# data de entrenamiento
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [17]:
# normalizar input
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

In [18]:
# Kfold para dividir la data de entrenamiento y la de test
kf = KFold(n_splits=3, shuffle=True, random_state=4305)

In [19]:
# crear modelos de Perceptrón multicapa
mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [(9, 9), (8, 8), (7, 7), (6, 6), (5, 5), (4, 4), (3, 3), (9, 9, 9), (8, 8, 8), (7, 7, 7), (6, 6, 6), (5, 5, 5), (4, 4, 4), (3, 3, 3)],
              'activation': ['relu'],
              'solver': ['adam'],
              'nesterovs_momentum' : [True],
              'momentum' : [0.9, 0.8, 0.7, 0.6, 0.5],                          
              'learning_rate_init': [0.001, 0.01, 0.1, 0.3, 0.5, 1, 2],              
              'learning_rate' : ['adaptive'],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [True]
             }

In [20]:
# crear una función para calcular el error
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean(np.square(np.subtract(np.log1p(y_true), np.log1p(y_pred)))))

In [21]:
# crear SearchCV
search_cv = RandomizedSearchCV(mlp, param_grid, scoring=make_scorer(rmsle, greater_is_better=False),
                   cv=kf, verbose=3, pre_dispatch='2*n_jobs')

In [ ]:
search_cv.fit(X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu 
[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu, score=-0.5213640050280353, total= 2.5min
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu, score=-0.822297718704767, total=  25.0s
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.9min remaining:    0.0s


[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.5, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(3, 3), early_stopping=False, activation=relu, score=-0.8226646007718277, total=  13.3s
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(5, 5), early_stopping=False, activation=relu 


C:\Users\jelus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log1p
  after removing the cwd from sys.path.


[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(5, 5), early_stopping=False, activation=relu, score=-0.48357849489497307, total=  27.9s
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(5, 5), early_stopping=False, activation=relu 
[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(5, 5), early_stopping=False, activation=relu, score=-0.8230808482304707, total=  23.6s
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1, learning_rate=adaptive, hidden_layer_sizes=(5, 5), early_stopping=False, activation=relu 
[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=1,

C:\Users\jelus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log1p
  after removing the cwd from sys.path.


[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.7, max_iter=1000, learning_rate_init=0.001, learning_rate=adaptive, hidden_layer_sizes=(6, 6, 6), early_stopping=False, activation=relu, score=-0.5023303250237445, total= 1.3min
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.9, max_iter=1000, learning_rate_init=0.3, learning_rate=adaptive, hidden_layer_sizes=(7, 7, 7), early_stopping=False, activation=relu 
[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.9, max_iter=1000, learning_rate_init=0.3, learning_rate=adaptive, hidden_layer_sizes=(7, 7, 7), early_stopping=False, activation=relu, score=-0.8189921511759177, total=  29.9s
[CV] warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.9, max_iter=1000, learning_rate_init=0.3, learning_rate=adaptive, hidden_layer_sizes=(7, 7, 7), early_stopping=False, activation=relu 
[CV]  warm_start=True, solver=adam, nesterovs_momentum=True, momentum=0.9, max_iter=1000, 

In [ ]:
# ver Top5 modelos
results = pd.DataFrame(search_cv.cv_results_).sort_values(by='mean_test_score', ascending=False)
results.head()

In [ ]:
search_cv.best_estimator_